In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

# We have to find Q - Matrix :
# Then we need to apply it to other problems

In [3]:
N = 50
L = 40
v0 = 20
episodes = 1000
learning_steps = 20
Q = np.ones((2,2))
gamma = 0.99
alpha_0 = 0.8
epsilon_0 = 0.995
D0 = 1
dt = 0.01
w0 = 13.3
tau_r = (1.10688**2) / (3*D0)
sigma_eff = 1.10688

In [4]:
Q

array([[1., 1.],
       [1., 1.]])

# Rows - states : grad position
# Columns - actions : w0
# ----------------------------------------

* Row :

   - 0 - concentration grad is to right of particle --- cross_prod = r * c = -

   - 1 - concentration grad is to left of particle --- cross_prod = r * c = +


* Column :

   - 0 - turn right - (-w0)

   - 1 - turn left  - (+w0)

In [5]:
# Reward for each particle is defined as the local concentration at the particle's location

In [6]:
# Hard sphere approximation :

In [7]:
def update_pos_phi_hardcore_periodic_box(pos, phi , omega , dt = dt , v0 = v0 ,D_t = D0 , D_r = tau_r , sigma_eff = sigma_eff , L = L):
    N = pos.shape[0]

    dphi = omega * dt + np.sqrt(2 * dt/ D_r) * np.random.randn(N)
    phi_new = (phi + dphi) % (2 * np.pi)

    direction = np.stack([np.cos(phi_new), np.sin(phi_new)], axis=1)
    displacement = v0 * dt * direction
    noise = np.sqrt(2 * D_t * dt) * np.random.randn(N, 2)
    proposed = pos + displacement + noise

    proposed = (proposed + L/2) % L - L/2

    pos_new = pos.copy()
    for i in range(N):
        accept = True
        for j in range(N):
            if i == j:
                continue
            delta = proposed[i] - pos_new[j]
            delta = delta - L * np.round(delta / L)
            dist = np.linalg.norm(delta)
            if dist < sigma_eff:
                accept = False
                break
        if accept:
            pos_new[i] = proposed[i]

    return pos_new, phi_new


In [8]:
# SPHERE TOUCHING APPROXIMATION :

In [9]:
def truncate_move_if_overlap(pos, disp, i, sigma_eff, L):
    N = pos.shape[0]
    max_lambda = 1.0
    for j in range(N):
        if i == j:
            continue
        rij = pos[j] - pos[i]
        rij = rij - L * np.round(rij / L)
        dist_sq = np.dot(rij, rij)
        if dist_sq < 1e-12:
            continue
        dv = disp[i]
        dv_sq = np.dot(dv, dv)
        rij_dv = np.dot(rij, dv)
        a = dv_sq
        b = 2 * rij_dv
        c = dist_sq - sigma_eff**2
        disc = b**2 - 4*a*c
        if disc < 0:
            continue
        sqrt_disc = np.sqrt(disc)
        lam1 = (-b - sqrt_disc) / (2*a)
        lam2 = (-b + sqrt_disc) / (2*a)
        if 0 <= lam1 <= 1:
            max_lambda = min(max_lambda, lam1)
        elif 0 <= lam2 <= 1:
            max_lambda = min(max_lambda, lam2)
    return disp[i] * max_lambda

def update_pos_phi_touching(pos, phi , omega , dt = dt , v0 = v0 ,D_t = D0 , D_r = tau_r , sigma_eff = sigma_eff , L = L):
    N = pos.shape[0]
    dphi = omega * dt + np.sqrt(2 * D_r * dt) * np.random.randn(N)
    phi_new = (phi + dphi) % (2 * np.pi)
    direction = np.stack([np.cos(phi_new), np.sin(phi_new)], axis=1)
    active_disp = v0 * dt * direction
    noise = np.sqrt(2 * D_t * dt) * np.random.randn(N, 2)
    disp = active_disp + noise
    pos_new = pos.copy()
    for i in range(N):
        new_disp = truncate_move_if_overlap(pos, disp, i, sigma_eff, L)
        pos_new[i] = pos[i] + new_disp
    pos_new = (pos_new + L/2) % L - L/2
    return pos_new, phi_new

In [10]:
def compute_concentration_gradient(positions , lam = 10):
    N = positions.shape[0]
    grad = np.zeros((N, 2), dtype=float)

    for k in range(N):
        for i in range(N):
            if i == k:
                continue
            r_vec = positions[k] - positions[i]
            r = np.linalg.norm(r_vec)
            if r > 0:
                exp_term = np.exp(-r / lam) / (4*np.pi)
                factor = exp_term * (-(1 + (r/lam)) / r**3)
                grad[k] += factor * r_vec

    return grad

In [11]:
def direction_gradient_cross_product(phi_ , grad):
    dir_x = np.cos(phi_)
    dir_y = np.sin(phi_)

    grad_x = grad[: , 0]
    grad_y = grad[: , 1]

    cross_z = dir_x * grad_y - dir_y * grad_x

    return (cross_z > 0).astype(int)

In [12]:
for ep in range(episodes):

    alpha = alpha_0 / (alpha_0 + ep)
    epsilon = epsilon_0 ** ep

    pos_ = (np.random.rand(N,2) - 0.5) * L
    phi_ = (np.random.rand(N)) * 2 * np.pi

    for step in range(learning_steps):

        grad_ = compute_concentration_gradient(pos_)
        states_s0 = direction_gradient_cross_product(phi_ , grad_)

        best = np.argmax(Q[states_s0] , axis = 1)
        rand = np.random.rand(N)
        a = np.where(rand < (1-epsilon) , best , np.random.randint(0,2,N))

        ws = np.where(a==0 , -w0 , w0)

        pos_ , phi_ = update_pos_phi_touching(pos_ , phi_ , ws)

        states_s1 = direction_gradient_cross_product(phi_ , grad_)

        reward = np.zeros(N)
        for k in range(N):
            for j in range(N):
                if k != j:
                    r_vec = pos_[k] - pos_[j]
                    r = np.linalg.norm(r_vec)
                    reward[k] += (np.exp(-r / 10) / (r + 1e-12)/(4 * np.pi))

        q_matrix_max = Q[states_s1].max(axis = 1)

        for i in range(N):
            Q[states_s0[i] , a[i]] += alpha * (reward[i] + gamma * (q_matrix_max[i]) - Q[states_s0[i] , a[i]])

    if ep%10 == 0:
        print(ep)
        print(Q , '---' , 'Act {' , Q[0].argmax() , Q[1].argmax() ,' }')

0
[[3.41800554 3.25356473]
 [3.2564115  3.17467773]] --- Act { 0 0  }
10
[[9.82237355 9.82972862]
 [9.97663776 9.98492459]] --- Act { 1 1  }
20
[[11.02773552 11.02016265]
 [11.09197297 11.08197686]] --- Act { 0 0  }
30
[[11.25137841 11.25434703]
 [11.23696089 11.23061858]] --- Act { 1 0  }
40
[[11.33217343 11.33745805]
 [11.38782323 11.38717925]] --- Act { 1 0  }
50
[[11.39843    11.402806  ]
 [11.34592067 11.350415  ]] --- Act { 1 1  }
60
[[11.48954587 11.47274452]
 [11.52774889 11.53185988]] --- Act { 0 1  }
70
[[11.48383484 11.48081596]
 [11.48753412 11.48403443]] --- Act { 0 0  }
80
[[11.38867507 11.38976026]
 [11.43328843 11.44709039]] --- Act { 1 1  }
90
[[11.43964944 11.43108059]
 [11.44637612 11.45320629]] --- Act { 0 1  }
100
[[11.71071891 11.70701583]
 [11.68433929 11.68030433]] --- Act { 0 0  }
110
[[11.73592938 11.73262234]
 [11.69994568 11.68632934]] --- Act { 0 0  }
120
[[11.93986575 11.91871762]
 [11.87616542 11.90427655]] --- Act { 0 1  }
130
[[11.84216196 11.8219396 ]


In [13]:
Q

array([[13.99816115, 13.80726417],
       [13.80444544, 13.98265994]])

In [14]:
s1 = Q[0].argmax()
s2 = Q[1].argmax()

In [15]:
print('State - 1 : ' , s1)
print('State - 2 : ' , s2)

State - 1 :  0
State - 2 :  1


# What does this say ?
* if the grad is at left - turn left - (+w0)
* if the grad is at right - turn right - (-w0)